## Figure Friday 2024 - W48
This is the **second** version and it has an improved layout for the dashboard app that consider some other callbacks and charts.

In [ ]:
"""Just importing modules"""
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc
import plotly.express as px
import plotly.io as pio
import pandas as pd
import numpy as np
from pathlib import Path

pio.templates.default = 'plotly_white'

# Original data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/Figure-Friday/refs/heads/main/2024/week-48/API_IT.NET.USER.ZS_DS2_en_csv_v2_2160.csv")

# metadata columns: ['Country Code', 'Region', 'IncomeGroup', 'SpecialNotes', 'TableName']
df_meta = pd.read_csv("https://raw.githubusercontent.com/plotly/Figure-Friday/refs/heads/main/2024/week-48/Metadata_Country_API_IT.NET.USER.ZS_DS2_en_csv_v2_2160.csv")
df_meta.dropna(subset='IncomeGroup', axis=0, inplace=True)

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## First Part to melt data and plot a line_chart by country
exclude_cols = ['Country Code', 'Indicator Name', 'Indicator Code']
df_melted = (pd.melt(
    df[df.columns.difference(exclude_cols, sort=False)],
    id_vars=['Country Name'],
    var_name='Year',
    value_name='Quantity'
))

df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')
# Drop rows where 'Year' is NaN (non-year columns) or 'Quantity' is NaN
df_melted = df_melted.dropna(subset=['Year', 'Quantity'])

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Second Part to merge with df_meta to add metadata
df_merged = (df.merge(df_meta, how='left', left_on='Country Code', right_on='Country Code'))
# row_to_drop = df_merged[df_merged['Code'].isna()].index.values
# df_merged.drop(index=row_to_drop, axis=0, inplace=True) # type: ignore

# Dropping columns with threshold 9 non-null
dff = (df_merged.dropna(axis=1, thresh=9))

## Melted with groupers
dff2 = (pd.melt(
    dff[dff.columns.difference(exclude_cols+['SpecialNotes', 'TableName'], sort=False)],
    id_vars=['IncomeGroup', 'Region', 'Country Name'],
    var_name='Year',
    value_name='Quantity'
))
dff2['Year'] = pd.to_numeric(dff2['Year'], errors='coerce')
dff2.dropna(axis=0, inplace=True, subset='Quantity')

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# This dff3 to plot choroplet map with 'Country Name'
dff3 = (pd.melt(
    dff[dff.columns.difference(exclude_cols+['IncomeGroup', 'SpecialNotes', 'TableName'], sort=False)],
    id_vars=['Region','Country Name'],
    var_name='Year',
    value_name='Quantity'
))
dff3['Year'] = pd.to_numeric(dff3['Year'], errors='coerce')
dff3.dropna(axis=0, inplace=True, subset='Quantity')

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# App Layout
# Initialize Dash app with Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

button_1 = dbc.Button(
      "Submit",
      id="submit-button",
      color="primary",
      n_clicks=0,
      className="mt-3")

# Layout definition
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H3("Global Internet Penetration Dashboard", className="text-start text-primary mb-4"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Select Countries:"),
            dmc.MultiSelect(
                id="country-dropdown",
                data=[{"label": country, "value": country} for country in df_melted['Country Name'].sort_values().unique()],
                maxSelectedValues=3,
                clearable=True,
                searchable=True,
                placeholder="Select up to 3 countries"
            )
        ], width=5),
        dbc.Col([
            html.Label("Select Year Range:"),
            dcc.RangeSlider(
                id="year-slider",
                min=dff2['Year'].min(),
                max=dff2['Year'].max(),
                step=1,
                tooltip={
                    "always_visible": True,
                    "template": "{value}",
                    "placement":'bottom',
                },
                marks={year: str(year) for year in range(dff2['Year'].min(), dff2['Year'].max(), 5)},
                value=[1990, 2023]
            )
        ], width=5),
        dbc.Col(button_1, 
            className="text-center",
            width=2
        )
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="time-series-chart"), width=12)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="region-treemap", figure={}), width=12)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="yoy-bar-chart", figure={}), width=6),
        dbc.Col(dcc.Graph(id="choropleth-map", figure={}), width=6)
    ]),
    dbc.Row([
        dbc.Col(html.Div(id="summary-statistics"), width=12)
    ])
], fluid=True)

# Callback to update the time-series chart only when submit button is clicked
@app.callback(
    Output("time-series-chart", "figure"),
    Input("submit-button", "n_clicks"),
    State("country-dropdown", "value"),
    State("year-slider", "value")
)
def update_time_series(n_clicks, selected_countries, year_range):
    if not selected_countries or not year_range:
        return px.line(title="Please select countries and year range.")
    
    filtered_df = df_melted[(df_melted["Year"] >= year_range[0]) & (df_melted["Year"] <= year_range[1])]
    filtered_df = filtered_df[filtered_df["Country Name"].isin(selected_countries)]
    
    fig = px.line(
        filtered_df,
        x="Year", y="Quantity",
        color="Country Name",
        line_shape="spline",
        title="Internet Penetration Over Time",
        markers=True,
        labels={"Quantity": "Internet Penetration (%)", 'Year':''}
    )
    fig.update_layout(hovermode='x')
    return fig

# Callback to update the treemap chart based on region selection
@app.callback(
    Output("region-treemap", "figure"),
    Input("year-slider", "value"),
    prevent_initial_call = True
)
def update_treemap(year_range):
    filtered_df = dff2[(dff2["Year"] >= year_range[0]) & (dff2["Year"] <= year_range[1])]
    agg_df = filtered_df.groupby("Region").agg({"Quantity": "sum"}).reset_index()
    total_quantity = agg_df["Quantity"].sum()
    agg_df["Proportion"] = (agg_df["Quantity"] / total_quantity) * 100

    fig = px.treemap(
        agg_df,
        path=["Region"],
        values="Proportion",
        title=f"Proportion of Global Internet Penetration by Continent from Y{year_range[0]}-Y{year_range[1]}",
        color="Proportion",
        color_continuous_scale="sunset_r",
        labels={"Proportion": "Global Share (%)"}
    )
    return fig

# Callback for updating the YoY growth bar chart
@app.callback(
    Output("yoy-bar-chart", "figure"),
    Input("submit-button", "n_clicks"),
    State("country-dropdown", "value"),
    State("year-slider", "value"),
    prevent_initial_call=True,
)
def update_yoy_chart(n_clicks, selected_countries, year_range):
    filtered_df = dff2[dff2["Country Name"].isin(selected_countries)]
    filtered_df3 = filtered_df.copy()
    filtered_df3["YoY Growth"] = filtered_df3.groupby("Country Name")["Quantity"].pct_change() * 100
    yoy_filtered = filtered_df3[(filtered_df3["Year"] >= year_range[0]) & (filtered_df3["Year"] <= year_range[1])]
    fig = px.bar(
        yoy_filtered, x="Year", y="YoY Growth", color="Country Name",
        barmode='group',
        title="Year-over-Year Growth"
    )
    return fig

# Callback for updating the choropleth map
@app.callback(
    Output("choropleth-map", "figure"),
    Input("submit-button", "n_clicks"),
    State("year-slider", "value"),
    prevent_initial_call=True,
)
def update_choropleth(n_clicks, year_range):
    filtered_df = dff3[dff3["Year"] == year_range[1]]
    fig = px.choropleth(
        filtered_df, locations="Country Name", locationmode="country names",
        scope='world', color="Quantity", color_continuous_scale=px.colors.sequential.YlOrBr,
        title=f"Internet Penetration by Region in Y{year_range[1]}"
    )
    return fig

if __name__ == "__main__":
    app.run(debug=True, port=8099, jupyter_mode='external')